# News Headlines Sentiment

Use the news api to pull the latest news articles for bitcoin and ethereum and create a DataFrame of sentiment scores for each coin. 

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [123]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient

%matplotlib inline

In [124]:
# Read your api key environment variable
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("news_api")


In [125]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=os.environ["news_api"])

In [134]:
from finvizfinance.screener.overview import Overview

foverview = Overview()

filters_dict = {
    'Performance':'Today Up',
    'Price':"Under $2", 
    'Relative Volume': "Over 3"
}
foverview.set_filter(filters_dict=filters_dict)

tickers = foverview.ScreenerView()
#tickers.sort_values(by='Volume', inplace=True, ascending=False)
tickers.set_index('Ticker', inplace=True )
tickers.head(8)

,Company,Sector,Industry,Country,Market Cap,P/E,Price,Change,Volume
Ticker,,,,,,,,,
CYRN,CYREN Ltd.,Technology,Software - Infrastructure,USA,67060000.0,None,1.24,0.0081,7089576.0
DNN,Denison Mines Corp.,Energy,Uranium,Canada,712000000.0,None,1.11,0.0472,152474141.0
LKCO,Luokung Technology Corp.,Communication Services,Internet Content & Information,China,361980000.0,None,1.59,0.3250,55653826.0
PT,Pintec Technology Holdings Limited,Financial,Credit Services,China,81750000.0,None,1.98,0.1445,3342644.0
RAVE,"RAVE Restaurant Group, Inc.",Consumer Cyclical,Restaurants,USA,31380000.0,None,1.70,0.0303,11882305.0
XPL,Solitario Zinc Corp.,Basic Materials,Other Industrial Metals & Mining,USA,53290000.0,None,0.94,0.0346,6559831.0


In [137]:
stock_headlines = [] 
for ticker in tickers.index: 
  #  print(tickers['Company'][ticker])
    stock_headlines.append( newsapi.get_everything(
    q=tickers['Company'][ticker],
    language='en',
    sort_by='relevancy'))
print(stock_headlines)

NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [67]:
# Create the Stock sentiment scores DataFrame
sentiments = []

for articles in stock_headlines['articles']:
    try:
        text = articles['content']
        results = analyzer.polarity_scores(text)
        compound = results['compound']
        pos = results['pos']
        neu = results['neu']
        neg = results['neg'] 

        #put data into sentiments
        sentiments.append({
            'text':text,
            'Compound':compound,
            'Positive':pos,
            'Negative':neg,
            'Neutral':neu})

    except AttributeError:
        pass

fininal_sentiments = pd.DataFrame(sentiments)
fininal_sentiments.head()

,Compound,Negative,Neutral,Positive,text
0,0.1027,0.063,0.864,0.073,"Weve got a full month of 2021 behind us now, a..."
1,0.3818,0.000,0.929,0.071,Investors have been fixated on growth companie...
2,0.7783,0.000,0.794,0.206,"Electric cars are growing in popularity, a tre..."
3,-0.2263,0.068,0.932,0.000,"CHENGDU, China, Feb. 10, 2021 /PRNewswire/ -- ..."
4,-0.2263,0.068,0.932,0.000,"CHENGDU, China, Feb. 8, 2021 /PRNewswire/ -- S..."


In [68]:
# Describe the Stock Sentiment
fininal_sentiments.describe()

,Compound,Negative,Neutral,Positive
count,6.000000,6.000000,6.000000,6.000000
mean,0.097317,0.044500,0.897167,0.058333
std,0.414459,0.034524,0.057266,0.080485
min,-0.226300,0.000000,0.794000,0.000000
25%,-0.226300,0.015750,0.880250,0.000000
50%,-0.061800,0.065500,0.930500,0.035500
75%,0.312025,0.068000,0.932000,0.072500
max,0.778300,0.068000,0.932000,0.206000


In [69]:
fininal_sentiments["Positive"].mean()

0.05833333333333333

In [70]:
fininal_sentiments=fininal_sentiments[fininal_sentiments["Positive"] >= 0.2]
fininal_sentiments.head()

,Compound,Negative,Neutral,Positive,text
2,0.7783,0.0,0.794,0.206,"Electric cars are growing in popularity, a tre..."


In [86]:
final_sentiment_stocks = df.loc[0]
final_sentiment_stocks.head()

Ticker                            AIHS
Company     Senmiao Technology Limited
Sector                       Financial
Industry               Credit Services
Country                          China
Name: 0, dtype: object